Data: 
- users.csv
- movies.csv
- ratings_new.csv

In [ ]:
# they want to find similar friends
# if they like A, you'll like A

In [1]:
import pandas as pd
import numpy as np

# Load the u.user file into a dataframe
# u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv('users.csv')
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [2]:
# Load the u.item file into a dataframe
# i_cols = ['movie_id', 'title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
#  'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
#  'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('movies.csv')
pd.set_option('display.max_columns', None)
movies.head()

,movie_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children\,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1-Jan-95,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1-Jan-95,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),1-Jan-95,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),1-Jan-95,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),1-Jan-95,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [3]:
# Remove all information except Movie ID and title
movies = movies[['movie_id', 'title']]
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [4]:
# Load the u.data file into a dataframe
# r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('ratings_new.csv')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
ratings.describe()

,user_id,movie_id,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [6]:
# Drop the timestamp column
ratings = ratings.drop('timestamp', axis=1)

In [7]:
ratings.shape

(100000, 3)

In [8]:
# Import the train_test_split function
from sklearn.model_selection import train_test_split

# Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = ratings.copy()
y = ratings['user_id']

# Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

In [9]:
# Import the mean_squared_error function
from sklearn.metrics import mean_squared_error

# Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [11]:
# Function to compute the RMSE score obtained on the testing set by a model
def score(cf_model):
    
    # Construct a list of user-movie tuples from the testing dataset
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    
    # Predict the rating for every user-movie tuple
    y_pred = np.array([cf_model(user, movie) for (user, movie) in id_pairs])
    
    # Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test['rating'])
    print(y_true)
    print(y_pred)
    
    # Return the final RMSE score
    return rmse(y_true, y_pred)

In [14]:
# Build the ratings matrix using pivot_table function
r_matrix = X_train.pivot_table(values='rating', index='user_id', columns='movie_id')
r_matrix.head()

movie_id  1     2     3     4     5     6     7     8     9     10    11    \
user_id                                                                      
1          5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0   2.0   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   4.0   
5          NaN   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

movie_id  12    13    14    15    16    17    18    19    20    21    22    \
user_id                                                                      
1          5.0   NaN   5.0   NaN   NaN   NaN   4.0   5.0   4.0   1.0   NaN   
2          NaN   4.0   4.0   NaN   NaN   NaN   NaN   3.0   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   NaN   NaN   NaN   NaN   4.0   NaN   NaN   NaN   3.0   NaN   

movie_id  23    24    25    26    27    28    29    30    31    32    33    \
user_id                                                                      
1          4.0   3.0   4.0   3.0   NaN   4.0   1.0   NaN   3.0   5.0   4.0   
2          NaN   NaN   4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   4.0   3.0   NaN   NaN   NaN   4.0   NaN   NaN   NaN   NaN   

movie_id  34    35    36    37    38    39    40    41    42    43    44    \
user_id                                                                      
1          NaN   1.0   2.0   NaN   3.0   4.0   NaN   2.0   NaN   4.0   5.0   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   5.0   NaN   NaN   

movie_id  45    46    47    48    49    50    51    52    53    54    55    \
user_id                                                                      
1          NaN   4.0   4.0   5.0   3.0   5.0   NaN   4.0   3.0   3.0   5.0   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

movie_id  56    57    58    59    60    61    62    63    64    65    66    \
user_id                                                                      
1          4.0   NaN   4.0   5.0   NaN   NaN   3.0   2.0   5.0   4.0   4.0   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0   NaN   NaN   NaN   

movie_id  67    68    69    70    71    72    73    74    75    76    77    \
user_id                                                                      
1          NaN   4.0   3.0   3.0   3.0   4.0   3.0   1.0   NaN   4.0   NaN   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   NaN   NaN   4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

movie_id  78    79    80    81    82    83    84    85    86    87    88    \
user_id                                                                      
1          1.0 

# Baseline

In [10]:
# Define the baseline model to always return 3
def baseline(user_id, movie_id):
    return 3.0

In [37]:
# user 2, movie 12
baseline(2,12)

3.0

In [12]:
# RMSE
score(baseline)

[4 2 3 ... 1 4 2]
[3. 3. 3. ... 3. 3. 3.]


1.2470926188539486

# User-Based Collaborative Filtering
## Mean

In [15]:
# User Based Collaborative Filter using Mean Ratings
def cf_user_mean(user_id, movie_id):
    
    # Check if movie_id exists in r_matrix
    if movie_id in r_matrix:
        
        # Compute the mean of all the ratings given to the movie
        mean_rating = r_matrix[movie_id].mean()
    else:
        
        # Default to a rating of 3.0 in the absence of any information
        mean_rating = 3.0  
    return mean_rating

In [27]:
# user 2, movie 12
cf_user_mean(2,12)

4.409326424870466

In [16]:
# RMSE
score(cf_user_mean)

[4 2 3 ... 1 4 2]
[4.         3.98058252 3.68571429 ... 3.71071429 3.90338164 3.        ]


1.0234701463131335

## Weighted Mean

In [17]:
r_matrix.head()

movie_id  1     2     3     4     5     6     7     8     9     10    11    \
user_id                                                                      
1          5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0   2.0   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   4.0   
5          NaN   3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

movie_id  12    13    14    15    16    17    18    19    20    21    22    \
user_id                                                                      
1          5.0   NaN   5.0   NaN   NaN   NaN   4.0   5.0   4.0   1.0   NaN   
2          NaN   4.0   4.0   NaN   NaN   NaN   NaN   3.0   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   NaN   NaN   NaN   NaN   4.0   NaN   NaN   NaN   3.0   NaN   

movie_id  23    24    25    26    27    28    29    30    31    32    33    \
user_id                                                                      
1          4.0   3.0   4.0   3.0   NaN   4.0   1.0   NaN   3.0   5.0   4.0   
2          NaN   NaN   4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   4.0   3.0   NaN   NaN   NaN   4.0   NaN   NaN   NaN   NaN   

movie_id  34    35    36    37    38    39    40    41    42    43    44    \
user_id                                                                      
1          NaN   1.0   2.0   NaN   3.0   4.0   NaN   2.0   NaN   4.0   5.0   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   5.0   NaN   NaN   

movie_id  45    46    47    48    49    50    51    52    53    54    55    \
user_id                                                                      
1          NaN   4.0   4.0   5.0   3.0   5.0   NaN   4.0   3.0   3.0   5.0   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

movie_id  56    57    58    59    60    61    62    63    64    65    66    \
user_id                                                                      
1          4.0   NaN   4.0   5.0   NaN   NaN   3.0   2.0   5.0   4.0   4.0   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0   NaN   NaN   NaN   

movie_id  67    68    69    70    71    72    73    74    75    76    77    \
user_id                                                                      
1          NaN   4.0   3.0   3.0   3.0   4.0   3.0   1.0   NaN   4.0   NaN   
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5          NaN   NaN   NaN   4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

movie_id  78    79    80    81    82    83    84    85    86    87    88    \
user_id                                                                      
1          1.0 

In [18]:
# Create a dummy ratings matrix with all null values imputed to 0
r_matrix_dummy = r_matrix.copy().fillna(0)
r_matrix_dummy.head()

movie_id  1     2     3     4     5     6     7     8     9     10    11    \
user_id                                                                      
1          5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0   2.0   
2          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0   0.0   
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   4.0   
5          0.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

movie_id  12    13    14    15    16    17    18    19    20    21    22    \
user_id                                                                      
1          5.0   0.0   5.0   0.0   0.0   0.0   4.0   5.0   4.0   1.0   0.0   
2          0.0   4.0   4.0   0.0   0.0   0.0   0.0   3.0   0.0   0.0   0.0   
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
5          0.0   0.0   0.0   0.0   0.0   4.0   0.0   0.0   0.0   3.0   0.0   

movie_id  23    24    25    26    27    28    29    30    31    32    33    \
user_id                                                                      
1          4.0   3.0   4.0   3.0   0.0   4.0   1.0   0.0   3.0   5.0   4.0   
2          0.0   0.0   4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
5          0.0   4.0   3.0   0.0   0.0   0.0   4.0   0.0   0.0   0.0   0.0   

movie_id  34    35    36    37    38    39    40    41    42    43    44    \
user_id                                                                      
1          0.0   1.0   2.0   0.0   3.0   4.0   0.0   2.0   0.0   4.0   5.0   
2          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
5          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   5.0   0.0   0.0   

movie_id  45    46    47    48    49    50    51    52    53    54    55    \
user_id                                                                      
1          0.0   4.0   4.0   5.0   3.0   5.0   0.0   4.0   3.0   3.0   5.0   
2          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
5          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

movie_id  56    57    58    59    60    61    62    63    64    65    66    \
user_id                                                                      
1          4.0   0.0   4.0   5.0   0.0   0.0   3.0   2.0   5.0   4.0   4.0   
2          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
5          0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   

movie_id  67    68    69    70    71    72    73    74    75    76    77    \
user_id                                                                      
1          0.0   4.0   3.0   3.0   3.0   4.0   3.0   1.0   0.0   4.0   0.0   
2          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
5          0.0   0.0   0.0   4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

movie_id  78    79    80    81    82    83    84    85    86    87    88    \
user_id                                                                      
1          1.0 

In [19]:
# Import cosine_score 
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [20]:
# Convert into pandas dataframe 
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)
cosine_sim.head()

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [21]:
# User Based Collaborative Filter using Weighted Mean Ratings
def cf_user_wmean(user_id, movie_id):
    
    # Check if movie_id exists in r_matrix
    if movie_id in r_matrix:
        
        # Get the similarity scores for the user in question with every other user
        sim_scores = cosine_sim[user_id]
        
        # Get the user ratings for the movie in question
        m_ratings = r_matrix[movie_id]
        
        # Extract the indices containing NaN in the m_ratings series
        idx = m_ratings[m_ratings.isnull()].index
        
        # Drop the NaN values from the m_ratings Series
        m_ratings = m_ratings.dropna()
        
        # Drop the corresponding cosine scores from the sim_scores series
        sim_scores = sim_scores.drop(idx)
        
        # Compute the final weighted mean
        wmean_rating = np.dot(sim_scores, m_ratings) / sim_scores.sum()
    
    else:
        # Default to a rating of 3.0 in the absence of any information
        wmean_rating = 3.0
    
    # pd.DataFrame(wmean_rating).to_csv('Data/wmean_rating.csv')
    return wmean_rating

In [28]:
# user 2, movie 12
cf_user_wmean(2,12)

4.399787171879637

In [23]:
# RMSE
score(cf_user_wmean)

[4 2 3 ... 1 4 2]
[4.0019218  3.93798639 3.81311742 ... 3.67256243 3.88113487 2.83484213]


1.0174483808407588

## Demographics

In [24]:
# Merge the original users dataframe with the training set 
merged_df = pd.merge(X_train, users)
merged_df.head()

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,889,684,2,24,M,technician,78704
1,889,279,2,24,M,technician,78704
2,889,29,3,24,M,technician,78704
3,889,190,3,24,M,technician,78704
4,889,232,3,24,M,technician,78704


In [25]:
# Compute the mean rating of every movie by gender
# 每个movie每个gender的rating mean
gender_mean = merged_df[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()
gender_mean

movie_id  sex
1         F      3.827586
          M      3.918919
2         F      3.230769
          M      3.228916
3         F      2.785714
                   ...   
1675      M      3.000000
1676      M      2.000000
1679      M      3.000000
1681      M      3.000000
1682      M      3.000000
Name: rating, Length: 3048, dtype: float64

In [26]:
# Set the index of the users dataframe to the user_id
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


### Gender

In [29]:
# Gender Based Collaborative Filter using Mean Ratings
def cf_gender(user_id, movie_id):
    
    # Check if movie_id exists in r_matrix (or training set)
    if movie_id in r_matrix:
        
        # Identify the gender of the user
        gender = users.loc[user_id]['sex']
        
        # Check if the gender has rated the movie
        if gender in gender_mean[movie_id]:
            
            # Compute the mean rating given by that gender to the movie
            gender_rating = gender_mean[movie_id][gender] 
            
        else:
            gender_rating = 3.0
    else:
        
        # Default to a rating of 3.0 in the absence of any information
        gender_rating = 3.0
        
    return gender_rating

In [30]:
# user 2, movie 12
cf_gender(2,12)

4.472222222222222

In [31]:
# RMSE
score(cf_gender)

[4 2 3 ... 1 4 2]
[3.96296296 3.31818182 3.71794872 ... 3.85714286 3.90853659 3.        ]


1.0330308800874282

In [32]:
# Compute the mean rating by gender and occupation
gen_occ_mean = merged_df[['sex', 'rating', 'movie_id', 'occupation']].pivot_table(
    values='rating', index='movie_id', columns=['occupation', 'sex'], aggfunc='mean')

gen_occ_mean.head()

occupation administrator           artist              doctor educator  \
sex                    F         M      F         M         M        F   
movie_id                                                                 
1                    4.0  4.222222   4.25  3.500000  3.666667     3.50   
2                    3.0  3.750000    NaN       NaN       NaN      NaN   
3                    3.5  2.500000    NaN       NaN       NaN     4.00   
4                    3.0  3.888889    NaN  4.666667  3.000000     2.75   
5                    4.0  2.333333    NaN       NaN       NaN     4.00   

occupation           engineer           entertainment           executive  \
sex                M        F         M             F         M         F   
movie_id                                                                    
1           3.923077      4.0  3.970588           5.0  3.222222       3.0   
2           3.250000      NaN  3.363636           NaN  3.000000       NaN   
3           2.500000      NaN  3.625000           NaN       NaN       NaN   
4           3.636364      NaN  3.555556           NaN  4.000000       NaN   
5           1.500000      NaN  2.666667           NaN  3.250000       NaN   

occupation           healthcare      homemaker     lawyer           librarian  \
sex                M          F    M         F   M      F         M         F   
movie_id                                                                        
1           4.066667   2.250000  4.0       NaN NaN    NaN  4.166667      4.00   
2           3.000000   2.666667  NaN       NaN NaN    NaN  5.000000      3.00   
3           2.000000        NaN  NaN       NaN NaN    NaN       NaN      2.50   
4           3.400000   3.000000  NaN       NaN NaN    NaN  4.500000      3.60   
5           4.500000   3.000000  NaN       NaN NaN    NaN       NaN      3.25   

occupation           marketing       none          other           programmer  \
sex                M         F     M    F    M         F         M          F   
movie_id                                                                        
1           3.571429       4.0  4.25  3.5  5.0  4.000000  4.107143        4.0   
2                NaN       NaN  3.00  5.0  4.0  3.666667  3.500000        NaN   
3                NaN       NaN  2.00  1.0  5.0  3.500000  3.250000        NaN   
4           3.000000       NaN  3.50  5.0  2.0  3.750000  2.888889        3.0   
5           4.000000       NaN  3.00  5.0  4.0  4.333333  3.600000        NaN   

occupation           retired           salesman           scientist            \
sex                M       F         M        F         M         F         M   
movie_id                                                                        
1           4.280000     NaN  3.666667      4.0  4.000000       3.5  3.888889   
2           3.000000     NaN  2.000000      NaN       NaN       NaN       NaN   
3           2.600000     NaN       NaN      NaN  1.000000       NaN       NaN   
4           3.615385     4.0  4.000000      4.0  3.666667       NaN  3.600000   
5           3.285714     NaN  3.500000      NaN       NaN       NaN  3.500000   

occupation   student           technician              writer            
sex                F         M          F         M         F         M  
movie_id                                                                 
1           3.833333  3.709091        4.0  4.200000  4.166667  3.142857  
2           2.333333  3.333333        NaN  2.714286  5.000000  2.666667  
3           2.000000  3.217391        NaN  4.000000       NaN  1.000000  
4           3.285714  3.724138        NaN  3.200000  4.250000  3.500000  
5           4.333333  3.272727        NaN  3.333333  4.000000  2.666667

In [86]:
# pd.DataFrame(gen_occ_mean).to_csv('data/gen_occ_mean.csv', index=True)

### Gender and Occupation

In [33]:
# Gender and Occupation Based Collaborative Filter using Mean Ratings
def cf_gen_occ(user_id, movie_id):
    
    # Check if movie_id exists in gen_occ_mean
    if movie_id in gen_occ_mean.index:
        
        # Identify the user
        user = users.loc[user_id]
        
        # Identify the gender and occupation
        gender = user['sex']
        occ = user['occupation']
        
        # Check if the occupation has rated the movie
        if occ in gen_occ_mean.loc[movie_id]:
            
            # Check if the gender has rated the movie
            if gender in gen_occ_mean.loc[movie_id][occ]:
                
                # Extract the required rating
                rating = gen_occ_mean.loc[movie_id][occ][gender]
                
                # Default to 3.0 if the rating is null
                if np.isnan(rating):
                    rating = 3.0
                
                return rating
            
    # Return the default rating    
    return 3.0

In [34]:
# user 2, movie 12
cf_gen_occ(2,12)

4.5

In [35]:
# RMSE
score(cf_gen_occ)

[4 2 3 ... 1 4 2]
[3.         4.         3.6        ... 4.05555556 3.77777778 3.        ]


1.1391976012043645

In [36]:
print('baseline:',score(baseline))
print()
print('cf_user_mean:',score(cf_user_mean))
print()
print('cf_user_wmean:',score(cf_user_wmean))
print()
print('cf_gender:',score(cf_gender))
print()
print('cf_gen_occ:',score(cf_gen_occ))

[4 2 3 ... 1 4 2]
[3. 3. 3. ... 3. 3. 3.]
baseline: 1.2470926188539486

[4 2 3 ... 1 4 2]
[4.         3.98058252 3.68571429 ... 3.71071429 3.90338164 3.        ]
cf_user_mean: 1.0234701463131335

[4 2 3 ... 1 4 2]
[4.0019218  3.93798639 3.81311742 ... 3.67256243 3.88113487 2.83484213]
cf_user_wmean: 1.0174483808407588

[4 2 3 ... 1 4 2]
[3.96296296 3.31818182 3.71794872 ... 3.85714286 3.90853659 3.        ]
cf_gender: 1.0330308800874282

[4 2 3 ... 1 4 2]
[3.         4.         3.6        ... 4.05555556 3.77777778 3.        ]
cf_gen_occ: 1.1391976012043645
